<h1> CS753 Assignment 4 Recommender Systems</h1>



<h4>1. Reading json files: https://www.geeksforgeeks.org/read-json-file-using-python/ </h4>

In [20]:
import json
import numpy as np

In [3]:
# read data
with open(r"C:\Users\ylive\OneDrive\Documents\compsci753\homework\train.json") as f:
    train_data = [json.loads(s) for s in f.readlines()]

with open(r"C:\Users\ylive\OneDrive\Documents\compsci753\homework\val.json") as f:
    val_data = [json.loads(s) for s in f.readlines()]

with open(r"C:\Users\ylive\OneDrive\Documents\compsci753\homework\test.json") as f:
    test_data = [json.loads(s) for s in f.readlines()]

all_data = train_data + val_data + test_data

In [5]:
# encode user and item
def encode_user_item(l:list):
    user_encode, item_encode = {}, {}
    i, j = 0, 0
    for r in l:
        if r['user_id'] not in user_encode:
            user_encode[r['user_id']] = i
            i += 1
        if r['business_id'] not in item_encode:
            item_encode[r['business_id']] = j
            j += 1
    return user_encode, item_encode

user_encode, item_encode = encode_user_item(all_data)

In [6]:
# get global bias
stars_tot = 0
for r in train_data:
    stars_tot += r['stars']
bg = stars_tot / len(train_data)

In [7]:
# get bias for users and items
def get_bias(l:list, bg:int):
    user_bias, item_bias = {}, {}
    for r in l:
        if r['user_id'] not in user_bias:
            user_bias[r['user_id']] = [r['stars'], 1]
        else:
            user_bias[r['user_id']][0] += r['stars']
            user_bias[r['user_id']][1] += 1

        if r['business_id'] not in item_bias:
            item_bias[r['business_id']] = [r['stars'], 1]
        else:
            item_bias[r['business_id']][0] += r['stars']
            item_bias[r['business_id']][1] += 1
    
    for u in user_bias.keys():
        user_bias[u] = user_bias[u][0]/user_bias[u][1] - bg

    for i in item_bias.keys():
        item_bias[i] = item_bias[i][0]/item_bias[i][1] - bg

    return user_bias, item_bias

user_bias, item_bias = get_bias(train_data, bg)

# if an user/item do not appear in training data, set the bias to 0
for r in all_data:
    if r['user_id'] not in user_bias:
        user_bias[r['user_id']] = 0
    if r['business_id'] not in item_bias:
        item_bias[r['business_id']] = 0

<h4>1. Task 1: Estimate the global bias bg, user specific bias b(user)i and item specific bias b(item)j on the training data. Report the global bg, the user specific bias of the user with user id= “b4aIMeXOx4cn3bjtdIOo6Q” , item specific bias of the business with business id = “7VQYoXk3Tc8EZeKuXeixeg”. [10 pts] </h4>

In [8]:
print(f'global bias: {bg}')
print(f'specific user bias, {user_bias["b4aIMeXOx4cn3bjtdIOo6Q"]}')
print(f'specific item bias, {item_bias["7VQYoXk3Tc8EZeKuXeixeg"]}')

global bias: 3.8290028774583242
specific user bias, -0.4605818248267455
specific item bias, -0.05122509968054656


<h4>2. Task 2: Implement the basic latent factor model without considering the bias: rij =qTi dot pj. Set the number of latent factors k = 8. Run Stochastic Gradient Descent (SGD) for 10 epoches with a fixed learning rate η = 0.01 and regularization hyperparameter λ1 = λ2 = 0.3. Report the RMSE on the training data for each epoch. [30 pts]</h4>


In [9]:
def LFM(l: list, k: int, seed=753):
    lr1, lr2 = .3, .3                                                               # regularization hyperparameter
    eta = .01                                                                       # learning rate                 
    max_epoch = 10                                                                  # number of epochs

    np.random.seed(seed)                                                            # user and item latent
    Q = np.random.randn(len(user_encode), k)
    P = np.random.randn(len(item_encode), k)

    print(f'K = {k}')
    
    for epoch in range(max_epoch):                                                  # for each epoch
        RSS = 0                                                                     # sum of squared residual
        for r in l:
            uid, iid = user_encode[r['user_id']], item_encode[r['business_id']]     # user and item id
            star = r['stars']                                                       # true rating
            starhat = Q[uid, :].dot(P[iid, :])                                      # estimated rating
            diff = star - starhat

            for f in range(k):                                                      # for each latent factor
                gu = -2 * diff * P[iid, f] + 2 * lr1 * Q[uid, f]                    # gradient of user
                gi = -2 * diff * Q[uid, f] + 2 * lr2 * P[iid, f]                    # gradient of item
                Q[uid, f] -= eta * gu                                               # update latent value
                P[iid, f] -= eta * gi
            RSS += diff**2

        print(f'epoch: {epoch}\tRMSE: {np.sqrt(RSS / len(l))}')
    return Q, P

In [10]:
# train latent factor model
_,_ = LFM(train_data, k=8)

K = 8
epoch: 0	RMSE: 3.3673747415210062
epoch: 1	RMSE: 1.6285505300063614
epoch: 2	RMSE: 1.2506534989921805
epoch: 3	RMSE: 1.1769729351730478
epoch: 4	RMSE: 1.1527096548402738
epoch: 5	RMSE: 1.1412240509945955
epoch: 6	RMSE: 1.1346082733364387
epoch: 7	RMSE: 1.130342583792446
epoch: 8	RMSE: 1.1273815666579499
epoch: 9	RMSE: 1.1252107693489868


<h4> 3. Task 3: Use SGD to train the latent factor model with different values of k in {4, 8, 16} and stop after 10 epoches. Report the RMSE for each value of k on the validation data (“val.json”). Pick the model that results in the best RMSE on the validation set and report its RMSE on the test data (“test.json”). [15 pts] </h4>

In [11]:
def evaluate(l: list, Q, P):
    RSS = 0                                                                         # sum of squared residual
    for r in l:
        uid, iid = user_encode[r['user_id']], item_encode[r['business_id']]         # user and item id
        star = r['stars']                                                           # true rating
        starhat = Q[uid, :].dot(P[iid, :])                                          # estimated rating
        RSS += (star-starhat)**2
    return np.sqrt(RSS / len(l))

In [12]:
# get latent for different number of factors (k)
latents = [LFM(train_data, k) for k in (4, 8, 16)]

K = 4
epoch: 0	RMSE: 3.2637850071345142
epoch: 1	RMSE: 1.6327983434261724
epoch: 2	RMSE: 1.253585355593168
epoch: 3	RMSE: 1.1857554853469283
epoch: 4	RMSE: 1.1655750540253302
epoch: 5	RMSE: 1.156592278359854
epoch: 6	RMSE: 1.1515745454527673
epoch: 7	RMSE: 1.1483821723447674
epoch: 8	RMSE: 1.1461709154332125
epoch: 9	RMSE: 1.144538340212954
K = 8
epoch: 0	RMSE: 3.3673747415210062
epoch: 1	RMSE: 1.6285505300063614
epoch: 2	RMSE: 1.2506534989921805
epoch: 3	RMSE: 1.1769729351730478
epoch: 4	RMSE: 1.1527096548402738
epoch: 5	RMSE: 1.1412240509945955
epoch: 6	RMSE: 1.1346082733364387
epoch: 7	RMSE: 1.130342583792446
epoch: 8	RMSE: 1.1273815666579499
epoch: 9	RMSE: 1.1252107693489868
K = 16
epoch: 0	RMSE: 3.5932500658968882
epoch: 1	RMSE: 1.6072781800169862
epoch: 2	RMSE: 1.2365328440116115
epoch: 3	RMSE: 1.1567503363128915
epoch: 4	RMSE: 1.1276708777824709
epoch: 5	RMSE: 1.1133400350125353
epoch: 6	RMSE: 1.1051162545849804
epoch: 7	RMSE: 1.099991918145948
epoch: 8	RMSE: 1.0966420518949864


In [14]:
# calculate RMSE on the validation set
rmse = np.array([evaluate(val_data, *l) for l in latents])
print(f'RMSE: {rmse}')

# get the LFM (without bias) with the best k on validation set
best_model = latents[np.argmin(rmse)]

# evaluate on the test set
print(f'The RMSE of the best LFM (without bias) on the test set: {evaluate(test_data, *best_model)}')

RMSE: [1.68821657 1.71790704 1.79128982]
The RMSE of the best LFM (without bias) on the test set: 1.7585057085341587


The RMSEs on the validation set are 1.69 1.72 1.79 when k is 4, 8, and 16, respectively. The best model on the validation set has k=4, and the RMSE on the test set is 1.76.

<h4> 4. Task 4: Incorporate the bias terms bg, b(user)i and b(item)jto the latent factor model:rij = bg + b(user)i + b(item)j + qTipj and learn the user bias and business bias from data.Initialize the user bias b(user)i and item bias terms b(item)j using the values computed in Task 1. Set the number of latent factors k = 8. Run SGD for 10 epoches with a fixed learning rate η = 0.01 and regularization hyperparameter λ1 = λ2 = λ3 = λ4 = 0.3.Report the RMSE on the training data for each epoch and the learned bias terms.[30 pts]</h4>

In [15]:
def LFM_bias(l: list, k: int, bg, user_bias, item_bias, seed=753):
    lr1, lr2, lr3, lr4 = .3, .3, .3, .3                                             # regularization hyperparameter
    eta = .01                                                                       # learning rate                 
    max_epoch = 10                                                                  # number of epochs

    user_bias = user_bias.copy()                                                    # leave the original bias unchanged
    item_bias = item_bias.copy()

    np.random.seed(seed)                                                            # user and item latent
    Q = np.random.randn(len(user_encode), k)
    P = np.random.randn(len(item_encode), k)

    print(f'K = {k}')

    for epoch in range(max_epoch):                                                  # for each epoch
        RSS = 0                                                                     # sum of squared residual
        for r in l:
            uid, iid = user_encode[r['user_id']], item_encode[r['business_id']]     # user and item id
            star = r['stars']                                                       # true rating
            starhat = bg + user_bias[r['user_id']] + \
                item_bias[r['business_id']] + Q[uid, :].dot(P[iid, :])              # estimated rating
            diff = star - starhat

            for f in range(k):                                                      # for each latent factor
                gu = -2 * diff * P[iid, f] + 2 * lr1 * Q[uid, f]                    # gradient of user
                gi = -2 * diff * Q[uid, f] + 2 * lr2 * P[iid, f]                    # gradient of item
                Q[uid, f] -= eta * gu                                               # update latent value
                P[iid, f] -= eta * gi
            user_bias[r['user_id']] -= eta * \
                (-2 * diff + 2 * lr3 * user_bias[r['user_id']])                     # update bias
            item_bias[r['business_id']] -= eta * \
                (-2 * diff + 2 * lr4 * item_bias[r['business_id']])
            RSS += diff**2

        print(f'epoch: {epoch}\tRMSE: {np.sqrt(RSS / len(l))}')
    return Q, P, user_bias, item_bias

In [16]:
Q8, P8, bu8, bi8 = LFM_bias(train_data, 8, bg, user_bias, item_bias)

K = 8
epoch: 0	RMSE: 1.5450943564667388
epoch: 1	RMSE: 1.1150599510290227
epoch: 2	RMSE: 1.0830770302123058
epoch: 3	RMSE: 1.0722505259961503
epoch: 4	RMSE: 1.0669333182315974
epoch: 5	RMSE: 1.063781408998576
epoch: 6	RMSE: 1.061699570666377
epoch: 7	RMSE: 1.060226776852334
epoch: 8	RMSE: 1.0591330653409852
epoch: 9	RMSE: 1.0582889577926036


In [17]:
# learned user-specific bias
print(f'user specific bias: {bu8["b4aIMeXOx4cn3bjtdIOo6Q"]}')

# learned item-specific bias
print(f'item specific bias: {bi8["7VQYoXk3Tc8EZeKuXeixeg"]}')

user specific bias: -0.2556560383594884
item specific bias: 0.04003793814146499


<h4> 5. Task 5: Similar to Task 3, find the best k on the validation set and apply the corresponding model to the test data. Compare the resulting test RMSE with Task3. [15 pts] </h4>

In [18]:
def evaluate_bias(l: list, bg, Q, P, bu, bi):
    RSS = 0                                                                         # sum of squared residual
    for r in l:
        uid, iid = user_encode[r['user_id']], item_encode[r['business_id']]         # user and item id
        star = r['stars']                                                           # true rating
        starhat = bg + user_bias[r['user_id']] + \
            item_bias[r['business_id']] + Q[uid, :].dot(P[iid, :])                  # estimated rating
        RSS += (star-starhat)**2
    return np.sqrt(RSS / len(l))

In [19]:
# get different LFM with respect to different k
latents_bias = [LFM_bias(train_data, k, bg, user_bias, item_bias) for k in (4, 8, 16)]


K = 4
epoch: 0	RMSE: 1.328367529467907
epoch: 1	RMSE: 1.1060855737583513
epoch: 2	RMSE: 1.088416228321949
epoch: 3	RMSE: 1.0823947299833754
epoch: 4	RMSE: 1.0793913193537505
epoch: 5	RMSE: 1.0775585785033288
epoch: 6	RMSE: 1.0762954207687971
epoch: 7	RMSE: 1.0753508621566465
epoch: 8	RMSE: 1.0746007339819574
epoch: 9	RMSE: 1.0739756802676534
K = 8
epoch: 0	RMSE: 1.5450943564667388
epoch: 1	RMSE: 1.1150599510290227
epoch: 2	RMSE: 1.0830770302123058
epoch: 3	RMSE: 1.0722505259961503
epoch: 4	RMSE: 1.0669333182315974
epoch: 5	RMSE: 1.063781408998576
epoch: 6	RMSE: 1.061699570666377
epoch: 7	RMSE: 1.060226776852334
epoch: 8	RMSE: 1.0591330653409852
epoch: 9	RMSE: 1.0582889577926036
K = 16
epoch: 0	RMSE: 1.9175274071555117
epoch: 1	RMSE: 1.1175162513150756
epoch: 2	RMSE: 1.0673835634072868
epoch: 3	RMSE: 1.0506052768372156
epoch: 4	RMSE: 1.0425802317735986
epoch: 5	RMSE: 1.0380734065725827
epoch: 6	RMSE: 1.0353444249287116
epoch: 7	RMSE: 1.0336433988732867
epoch: 8	RMSE: 1.0325867857937394


In [21]:
evaluate_bias(val_data, bg, Q8, P8, bu8, bi8)

1.2972461350179727

In [22]:
# calculate RMSE on the validation set
rmse_bias = np.array([evaluate_bias(val_data, bg, *l) for l in latents_bias])
print(f'RMSE: {rmse_bias}')

# get the LFM (with bias) with the best k on validation set
best_model_bias = latents_bias[rmse_bias.argmin()]

# evaluate on the test set
print(f'The RMSE of the best LFM (with bias) on the test set: {evaluate_bias(test_data, bg, *best_model_bias)}')

RMSE: [1.28254049 1.29724614 1.32627313]
The RMSE of the best LFM (with bias) on the test set: 1.2749177150868805


The RMSEs on the validation set are 1.28, 1.30 1.33 when k is 4, 8, and 16, respectively. The best model on the validation set has k=4, and the RMSE on the test set is 1.27.

Compare the resulting RMSE with Task 3, the RMSE decreases from 1.76 to 1.27 (on the test set).